<a href="https://colab.research.google.com/github/KritikaPantha/Spotify_analysis/blob/main/spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [6]:
albums_df = spark.read.csv("/content/sample_data/albums.csv", header=True, inferSchema=True)

In [7]:
artists_df = spark.read.csv("/content/sample_data/artists.csv", header=True, inferSchema=True)

In [8]:
track_df = spark.read.csv("/content/sample_data/track.csv", header=True, inferSchema=True)

In [9]:
albums_df.createOrReplaceTempView("albums")

In [10]:
artists_df.createOrReplaceTempView("artists")

In [11]:
track_df.createOrReplaceTempView("track")

In [13]:
#top ten most popular tracks
spark.sql("""
SELECT a.track_name,t.track_id, a.album_name, t.track_popularity, ar.name AS artist_name
FROM track t
JOIN albums a ON t.track_id = a.track_id
JOIN artists ar ON a.artist_id = ar.id
ORDER BY t.track_popularity DESC
LIMIT 10
""").show()

+--------------------+--------------------+--------------------+----------------+-------------------+
|          track_name|            track_id|          album_name|track_popularity|        artist_name|
+--------------------+--------------------+--------------------+----------------+-------------------+
|Wonderful Christm...|1SV1fxF65n9NhRHp3...|Mccartney Ii (Spe...|              90|     Paul McCartney|
|Calm Down (With S...|0WtM2NBVQNNJLh6sc...|Calm Down (With S...|              90|               Rema|
|   Say Yes To Heaven|6GGtHZgBycCgGBUhZ...|   Say Yes To Heaven|              89|       Lana Del Rey|
|                Numb|2nLtzopw4rPReszdY...|             Meteora|              89|        Linkin Park|
|               Creep|70LcF31zb1H0PyJoS...|         Pablo Honey|              89|          Radiohead|
|    Treat You Better|3QGsuHI8jO1Rx4JWL...|          Illuminate|              88|       Shawn Mendes|
| Young And Beautiful|2nMeu6UenVvwUktBC...| Young And Beautiful|              88| 

In [18]:
#albumns with most tracks
spark.sql("""
SELECT a.album_name, COUNT(t.track_id) AS track_count
FROM albums a
JOIN track t ON a.track_id = t.track_id
GROUP BY a.album_name
ORDER BY track_count DESC
LIMIT 10
""").show()

+--------------------+-----------+
|          album_name|track_count|
+--------------------+-----------+
|             Nirvana|        977|
|              Action|        527|
|          Red Velvet|        434|
|              Exodus|        433|
|              Future|        368|
|               India|        347|
|Bach: St. John Pa...|        332|
|               Blink|        307|
|                 Her|        269|
|Bach, J.S.: St. J...|        239|
+--------------------+-----------+



In [27]:
#total number of tracks per album
spark.sql("""
SELECT album_name, COUNT(track_id) AS track_count
FROM albums
GROUP BY album_name
ORDER BY track_count DESC
LIMIT 10
""").show()

+--------------------+-----------+
|          album_name|track_count|
+--------------------+-----------+
|             Nirvana|        977|
|              Action|        527|
|          Red Velvet|        434|
|              Exodus|        433|
|              Future|        368|
|               India|        347|
|Bach: St. John Pa...|        332|
|               Blink|        307|
|                 Her|        269|
|Bach, J.S.: St. J...|        239|
+--------------------+-----------+



In [28]:
#average track popularity and total tracks per artist and genre
spark.sql ("""
SELECT
ar.name AS artist_name,
ar.genre,
ROUND(AVG(t.track_popularity), 2) AS avg_track_popularity,
COUNT(t.track_id) AS total_tracks
FROM track t
JOIN albums a ON t.track_id = a.track_id
JOIN artists ar ON a.artist_id = ar.id
GROUP BY ar.name, ar.genre
ORDER BY avg_track_popularity DESC
LIMIT 10
""").show()

+-----------------+------------------+--------------------+------------+
|      artist_name|             genre|avg_track_popularity|total_tracks|
+-----------------+------------------+--------------------+------------+
|             Rema|         afrobeats|                90.0|           1|
|         NewJeans|             k-pop|                77.0|           6|
|       Young Nudy|           pluggnb|                76.0|           1|
|    Jacob Collier|uk alternative pop|                72.0|           1|
|Sabrina Carpenter|               pop|                70.0|           1|
|           Camilo|     colombian pop|                70.0|           1|
|      Holly Macve|   british country|                69.0|           1|
|     Proyecto Uno|       latin house|                69.0|           1|
|      Sofía Reyes|         latin pop|                69.0|           1|
|      Angel Ureta|              NULL|                69.0|           1|
+-----------------+------------------+-------------

In [29]:
#top 10 artists by followers
spark.sql("""
SELECT name, followers, artist_popularity
FROM artists
ORDER BY followers DESC
LIMIT 10
""").show()

+-------------+---------+-----------------+
|         name|followers|artist_popularity|
+-------------+---------+-----------------+
|   Ed Sheeran|115998928|               87|
| Taylor Swift| 95859165|              100|
|Ariana Grande| 95710972|               88|
|Billie Eilish| 89996504|               87|
|        Drake| 83298497|               95|
|       Eminem| 79891173|               89|
|    Bad Bunny| 77931484|               95|
|   The Weeknd| 75945958|               93|
|Justin Bieber| 75112165|               88|
|          BTS| 71720409|               85|
+-------------+---------+-----------------+

